## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## Reading data

Note that we dismiss Canary Islands, Balearic Islands Ceuta and Melilla because they are not considered in the demand data 

In [2]:
cities = pd.read_csv('cities_unified.csv', header = 0)
cities = cities[(cities['Provincias'] != 'Melilla') & (cities['Provincias'] != 'Ceuta') & (cities['Provincias'] != 'Santa Cruz de Tenerife') & (cities['Provincias'] != 'Palmas, Las') & (cities['Provincias'] != 'Balears, Illes')]
cities.reset_index(drop=True, inplace = True)
cities.index=cities['Codigos']
cities.tail()

,Codigos,Provincias,Latitud,Longitud,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Codigos,,,,,,,,,,,,,,
47,47,Valladolid,41.634126,-4.847191,529157,526288,523679,521130,519851,519546,520649,519361,517975,521333
48,48,Bizkaia,43.237680,-2.852600,1151905,1148775,1147576,1148302,1149628,1152651,1159443,1154334,1149344,1153282
49,49,Zamora,41.727174,-5.980539,185432,183436,180406,177404,174549,172539,170588,168725,167215,166927
50,50,Zaragoza,41.620365,-1.064497,960111,956006,950507,953486,954811,964693,972528,967452,966438,979365
0,0,TOTAL ESPAÑA,39.695119,-3.202023,46771341,46624382,46557008,46572132,46722980,47026208,47450795,47385107,47475420,48085361


In [3]:
ipc = pd.read_csv('IPC.csv', header = 0)
ipc = ipc[(ipc['Provincia'] != 'Melilla') & (ipc['Provincia'] != 'Ceuta') & (ipc['Provincia'] != 'Santa Cruz de Tenerife') & (ipc['Provincia'] != 'Palmas, Las') & (ipc['Provincia'] != 'Balears, Illes')]
ipc.reset_index(drop=True, inplace = True)
ipc.tail()

,Codigo,Provincia,1-2014,2-2014,3-2014,4-2014,5-2014,6-2014,7-2014,8-2014,...,3-2023,4-2023,5-2023,6-2023,7-2023,8-2023,9-2023,10-2023,11-2023,12-2023
43,47,Valladolid,93498,93451,93566,94419,94369,94419,93600,93776,...,111670,112360,112194,112770,112780,113328,113457,114086,113459,113537
44,48,Bizkaia,92529,92402,92640,93555,93525,93561,92738,92900,...,111128,111660,111744,112665,112700,113205,113485,113534,113198,113152
45,49,Zamora,93284,93234,93511,94390,94414,94352,93307,93525,...,113086,113937,113791,114536,114620,115011,115115,115517,114885,115055
46,50,Zaragoza,93588,93570,93708,94676,94661,94614,93792,93965,...,110967,111417,111269,111873,112310,112893,112907,113042,112828,112710
47,0,TOTAL ESPAÑA,93373,93333,93541,94373,94395,94421,93533,93681,...,111111,111773,111719,112354,112544,113149,113348,113676,113280,113308


In [4]:
ipc.columns

Index(['Codigo', 'Provincia', '1-2014', '2-2014', '3-2014', '4-2014', '5-2014',
       '6-2014', '7-2014', '8-2014',
       ...
       '3-2023', '4-2023', '5-2023', '6-2023', '7-2023', '8-2023', '9-2023',
       '10-2023', '11-2023', '12-2023'],
      dtype='object', length=122)

In [5]:
demanda_meteo = pd.read_csv('demanda_meteo.csv', header = 0)
demanda_meteo['Datetime'] = pd.to_datetime(demanda_meteo['Datetime'])
demanda_meteo.head()

,Datetime,Demanda Real,Demanda Prevista,Demanda Programada,Precio,temperature_2m_1,direct_normal_irradiance_instant_1,wind_speed_100m_1,temperature_2m_2,direct_normal_irradiance_instant_2,...,temperature_2m_51,direct_normal_irradiance_instant_51,wind_speed_100m_51,temperature_2m_52,direct_normal_irradiance_instant_52,wind_speed_100m_52,temperature_2m_0,direct_normal_irradiance_instant_0,wind_speed_100m_0,direct_normal_irradiance_instant_
0,2014-01-01 00:00:00,24301.66667,23878.33333,24099.0,37.21,7.4985,0.0,16.099690,7.874,0.0,...,11.714001,0.0,17.339897,10.032000,0.0,11.659777,5.791485,0.0,18.565381,0.0
1,2014-01-01 01:00:00,23365.33333,23322.00000,22269.0,30.57,7.0485,0.0,17.388409,7.724,0.0,...,11.614000,0.0,18.000000,9.731999,0.0,10.661107,5.709594,0.0,18.443398,0.0
2,2014-01-01 02:00:00,21672.50000,21711.16667,21136.0,26.30,7.3985,0.0,19.319628,7.574,0.0,...,11.564000,0.0,17.651016,9.932000,0.0,11.212135,5.620734,0.0,18.401250,0.0
3,2014-01-01 03:00:00,20192.00000,20230.33333,19911.0,26.18,7.3485,0.0,20.140705,7.424,0.0,...,11.464001,0.0,18.430452,10.432000,0.0,12.620554,5.597380,0.0,18.198554,0.0
4,2014-01-01 04:00:00,19220.66667,19215.33333,19060.0,22.40,7.4985,0.0,21.995380,7.374,0.0,...,11.464001,0.0,18.161278,10.582000,0.0,14.058450,5.656951,0.0,18.100382,0.0


In [6]:
festivos = pd.read_csv('Festivos.csv', header = 0)
festivos['Day'] = pd.to_datetime(festivos['Day'])
festivos = festivos.drop(['Melilla', 'Ceuta', 'Santa Cruz de Tenerife', 'Palmas, Las', 'Balears, Illes'], axis = 1)
festivos.head()

,Day,Day of the week,"Coruña, A",Albacete,Alicante/Alacant,Almería,Araba/Álava,Asturias,Badajoz,Barcelona,...,Soria,Tarragona,Teruel,Toledo,Valencia/València,Valladolid,Zamora,Zaragoza,Ávila,sumafest
0,2014-01-01,X,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.000000
1,2014-01-02,J,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.019659
2,2014-01-03,V,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,2014-01-04,S,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,2014-01-05,D,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000


In [7]:
pib = pd.read_csv('PIB.csv', header = 0)
pib.head()

,Codigo,Provincia,1-2014,2-2014,3-2014,4-2014,5-2014,6-2014,7-2014,8-2014,...,3-2023,4-2023,5-2023,6-2023,7-2023,8-2023,9-2023,10-2023,11-2023,12-2023
0,0,TOTAL ESPAÑA,925468,925468,925468,969598,969598,969598,960418,960418,...,1099141,1139036,1139036,1139036,1127405,1127405,1127405,1171293,1171293,1171293


## X and Y creation and definition of the metrics

In [8]:
ndays_base = 28

In [9]:
dias_semana = {0:'L', 1:'M', 2:'X', 3:'J', 4:'V', 5:'S', 6:'D'}

def build_df_diario (n=ndays_base):
    df = {'Datetime':[], 'Day':[], 'Month':[], 'Week':[],'F':[], 'L':[], 'M':[], 'X':[], 'J':[], 'V':[], 'S':[], 'IPC':[]}
    for i in range (len(festivos)):
        aux = {}
        aux['Datetime']=festivos['Day'][i]
        aux['Day'] = aux['Datetime'].day
        aux['Month'] = aux['Datetime'].month
        aux['Week'] = aux['Datetime'].week
        fest = festivos['sumafest'][i]
        aux['F']=fest
        for j in dias_semana.keys():
            aux[dias_semana[j]]=0
        aux[festivos['Day of the week'][i]] = 1-fest
        aux['IPC']=ipc[str(festivos['Day'][i].month)+'-'+str(festivos['Day'][i].year)][47]        
        for name in df.keys():
            df[name].append(aux[name])
        
            
    df = pd.DataFrame(df)
    return df
    
df_diario = build_df_diario()
df_diario    

,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC
0,2014-01-01,1,1,1,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,93373
1,2014-01-02,2,1,1,0.019659,0.0,0.0,0.0,0.980341,0.0,0.0,93373
2,2014-01-03,3,1,1,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,93373
3,2014-01-04,4,1,1,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,93373
4,2014-01-05,5,1,1,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,93373
...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2023-12-27,27,12,52,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,113308
3648,2023-12-28,28,12,52,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,113308
3649,2023-12-29,29,12,52,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,113308
3650,2023-12-30,30,12,52,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,113308


In [10]:
def build_base_diaria (df,n=ndays_base):
#     base = {'Day':[], 'F':[], 'L':[], 'M':[], 'X':[], 'J':[], 'V':[], 'S':[], 'D':[], 'IPC':[], 'PIB':[]}
    base = {'Datetime':[], 'Day':[], 'Month':[],'Week':[], 'F':[], 'L':[], 'M':[], 'X':[], 'J':[], 'V':[], 'S':[], 'IPC':[]}
    for i in range (len(festivos)):
        if i-n-2 <= 0:
            continue
        aux = {}
        aux['Datetime']=df['Datetime'][i]
        aux['Day'] = aux['Datetime'].day
        aux['Month'] = aux['Datetime'].month
        aux['Week'] = aux['Datetime'].week
        fest = df['F'][i]
        aux['F']=fest
        for j in dias_semana.keys():
            aux[dias_semana[j]]=0
        if festivos['Day of the week'][i] != 'D':
            aux[festivos['Day of the week'][i]] = (1-fest)/(np.mean(df['F'][i-(n+1):i]))
        aux['IPC']=df['IPC'][i]/(np.mean(df['IPC'][i-(n+1):i]))
        
        for name in base.keys():
            base[name].append(aux[name])
        
            
    base = pd.DataFrame(base)
    return base
    
base_diaria = build_base_diaria(df_diario)
threshold = base_diaria['Datetime'][0]
print (threshold)
# base_diaria = df_diario[df_diario['Datetime']>=threshold].reset_index(drop=True)
base_diaria

2014-02-01 00:00:00


,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC
0,2014-02-01,1,2,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.675067,0.999572
1,2014-02-02,2,2,5,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.999586
2,2014-02-03,3,2,6,0.0,4.746262,0.000000,0.000000,0.000000,0.000000,0.000000,0.999601
3,2014-02-04,4,2,6,0.0,0.000000,5.675067,0.000000,0.000000,0.000000,0.000000,0.999616
4,2014-02-05,5,2,6,0.0,0.000000,0.000000,7.055838,0.000000,0.000000,0.000000,0.999631
...,...,...,...,...,...,...,...,...,...,...,...,...
3616,2023-12-27,27,12,52,0.0,0.000000,0.000000,3.848015,0.000000,0.000000,0.000000,1.000026
3617,2023-12-28,28,12,52,0.0,0.000000,0.000000,0.000000,3.848015,0.000000,0.000000,1.000017
3618,2023-12-29,29,12,52,0.0,0.000000,0.000000,0.000000,0.000000,3.855166,0.000000,1.000009
3619,2023-12-30,30,12,52,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.855166,1.000000


Dividimos la temperatura en 5 franjas, por clima y localización.

### Clima Mediterráneo Sur:
- Andalucía: Sevilla, Málaga, Cádiz, Huelva, Córdoba, Granada, Jaén, Almería
- Región de Murcia: Murcia

### Clima Mediterráneo Norte:
- Comunidad Valenciana: Valencia, Alicante, Castellón de la Plana
- Cataluña: Barcelona, Tarragona, Girona, Lleida

### Clima Oceánico:
- Galicia: A Coruña, Lugo, Ourense, Pontevedra
- Asturias: Oviedo
- Cantabria: Santander
- País Vasco: Bilbao, San Sebastián, Vitoria-Gasteiz

### Clima Semiárido:
- Castilla-La Mancha: Albacete, Toledo, Cuenca, Ciudad Real
- Comunidad de Madrid: Madrid
- Extremadura: Badajoz, Cáceres

### Clima Continental:
- Castilla y León: Valladolid, Zamora, Salamanca, Ávila, Segovia
- La Rioja: Logroño
- Navarra: Pamplona
- Aragón: Huesca, Zaragoza, Teruel

In [11]:
zonas_temp = {'Ocean': ['Araba/Álava', 'Bizkaia', 'Gipuzkoa', 'Cantabria', 'Asturias', 'Pontevedra', 'Coruña, A', 'Lugo', 'Ourense'],
             'Med_N': ['Barcelona', 'Girona', 'Lleida', 'Tarragona', 'Castellón/Castelló', 'Alicante/Alacant', 'Valencia/València'],
             'Med_S': ['Almería', 'Cádiz', 'Huelva', 'Sevilla', 'Granada', 'Málaga', 'Córdoba', 'Jaén', 'Murcia'],
             'Arido': ['Albacete', 'Badajoz', 'Cáceres', 'Toledo', 'Cuenca', 'Ciudad Real', 'Madrid'],
             'Cont': ['Ávila', 'León', 'Palencia', 'Salamanca', 'Zamora', 'Valladolid', 'Segovia', 'Burgos', 'Soria', 'Rioja, La', 'Zaragoza', 'Navarra', 'Huesca', 'Teruel']}

# Assuming you have a DataFrame called df with columns 'Provincias' and 'Codigos'
# Create a dictionary to map each province to its corresponding 'Codigos' value
provincia_codigos_dict = cities.set_index('Provincias')['Codigos'].to_dict()

# Now, let's create a new dictionary where we replace province names with their corresponding Codigos
zonas_codigos = {}
for zona, provincias in zonas_temp.items():
    zonas_codigos[zona] = [provincia_codigos_dict[provincia] for provincia in provincias]

zonas_codigos

{'Ocean': [1, 48, 20, 39, 33, 36, 15, 27, 32],
 'Med_N': [8, 17, 25, 43, 12, 3, 46],
 'Med_S': [4, 11, 21, 41, 18, 29, 14, 23, 30],
 'Arido': [2, 6, 10, 45, 16, 13, 28],
 'Cont': [5, 24, 34, 37, 49, 47, 40, 9, 42, 26, 50, 31, 22, 44]}

In [12]:
zonas_aux = {'Med_Arido':zonas_temp['Med_N'] + zonas_temp['Med_S'] +zonas_temp['Arido'], 'Ocean_Cont':zonas_temp['Cont'] + zonas_temp['Ocean']}
zonas_temp = zonas_aux
zonas_aux = {'Med_Arido':zonas_codigos['Med_N'] + zonas_codigos['Med_S'] +zonas_codigos['Arido'], 'Ocean_Cont':zonas_codigos['Cont'] + zonas_codigos['Ocean']}
zonas_codigos = zonas_aux

In [13]:
def build_df_horario(limit_temp = 15, n=ndays_base):
    total = {}
    for h in range (24):
        auxdemanda = demanda_meteo[demanda_meteo['Datetime'].dt.hour == h]
        auxdemanda.reset_index(inplace=True)
#         df = {'Datetime':[], 'Demanda': [], 'Frío': [], 'Calor':[], 'Radiacion':[], 'Viento':[], 'Precio':[]}
#         df = {'Datetime':[], 'Demanda': [], 'Temp':[], 'Radiacion':[], 'Viento':[], 'Precio':[]}
#         df = {'Datetime':[], 'Demanda': [], 'Temp_Ocean':[], 'Temp_Med_N':[], 'Temp_Med_S':[], 'Temp_Arido':[], 'Temp_Cont':[], 'Radiacion':[], 'Viento':[], 'Precio':[]}
#         df = {'Datetime':[], 'Demanda': [], 'Temp_Ocean':[], 'Temp_Med_N':[], 'Temp_Med_S':[], 'Temp_Arido':[], 'Temp_Cont':[], 'Rad_Ocean':[], 'Rad_Med_N':[], 'Rad_Med_S':[], 'Rad_Arido':[], 'Rad_Cont':[], 'Viento':[], 'Precio':[]}
#         df = {'Datetime':[], 'Demanda': [], 'Temp_Ocean':[], 'Temp_Resto':[],'Rad_Ocean':[], 'Rad_Resto':[], 'Viento':[], 'Precio':[]}
        df = {'Datetime':[], 'Demanda': [], 'Temp_Ocean_Cont':[], 'Temp_Med_Arido':[],'Rad_Ocean_Cont':[], 'Rad_Med_Arido':[], 'Viento':[], 'Precio':[]}

        for i in range (len(auxdemanda)):
            aux = {}
            aux['Datetime'] = auxdemanda['Datetime'][i]
            aux['Demanda'] = auxdemanda['Demanda Real'][i]
#             if auxdemanda['temperature_2m_0'][i] <= limit_temp:
#                 aux['Frío'] = limit_temp - auxdemanda['temperature_2m_0'][i]
#                 aux['Calor'] = 0
                
#             else:
#                 aux['Calor'] = auxdemanda['temperature_2m_0'][i] - limit_temp
#                 aux['Frío'] = 0 

#             aux['Temp'] = auxdemanda['temperature_2m_0'][i]
                
            for zone in zonas_codigos.keys():
                sumapob = 0
                sumarad = 0
                sumatemp = 0
                for code in zonas_codigos[zone]:
                    stemp = 'temperature_2m_'+str(code)
                    srad = 'direct_normal_irradiance_instant_'+str(code)
                    sumatemp +=auxdemanda[stemp][i]*cities[str(auxdemanda['Datetime'][i].year)][code]
                    sumarad +=auxdemanda[srad][i]*cities[str(auxdemanda['Datetime'][i].year)][code]
                    sumapob += cities[str(auxdemanda['Datetime'][i].year)][code]
                aux['Temp_'+zone] = sumatemp/sumapob
                aux['Rad_'+zone] = sumarad/sumapob
#             aux['Radiacion'] = auxdemanda['direct_normal_irradiance_instant_0'][i]
            aux['Viento'] =  auxdemanda['wind_speed_100m_0'][i]
            aux['Precio'] =  auxdemanda['Precio'][i]
            
            
            for name in df.keys():
                df[name].append(aux[name])
        
        df = pd.DataFrame(df)
        total[h] = df
#         print (total[h]['Datetime'][0])
    return total

df_horario = build_df_horario()
df_horario[0]

,Datetime,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-01-01,24301.66667,6.236950,4.762810,0.0,0.0,18.565381,37.21
1,2014-01-02,23484.00000,9.201634,8.185117,0.0,0.0,24.776614,22.13
2,2014-01-03,26316.50000,11.000175,10.579728,0.0,0.0,23.791944,29.39
3,2014-01-04,26518.50000,8.642548,9.609179,0.0,0.0,21.050676,36.54
4,2014-01-05,26163.16667,3.840441,5.714173,0.0,0.0,28.805064,18.77
...,...,...,...,...,...,...,...,...
3647,2023-12-27,23764.08333,2.214768,3.441935,0.0,0.0,10.034081,83.06
3648,2023-12-28,24105.83333,4.735265,3.933591,0.0,0.0,10.283692,101.01
3649,2023-12-29,24181.41667,6.130193,4.762047,0.0,0.0,10.691312,106.83
3650,2023-12-30,24131.16667,6.216880,5.628986,0.0,0.0,8.431629,93.55


In [15]:
def build_base_horaria(df,limit_temp = 15, n=ndays_base):
    total = {}
    for h in range (24):
        auxdemanda = df[h]
#         base = {'Datetime':[], 'Demanda': [], 'Frío': [], 'Calor':[], 'Radiacion':[], 'Viento':[], 'Precio':[]}
#         base = {'Datetime':[], 'Demanda': [], 'Temp':[], 'Radiacion':[], 'Viento':[], 'Precio':[]}
#         base = {'Datetime':[], 'Demanda': [], 'Temp_Ocean':[], 'Temp_Med_N':[], 'Temp_Med_S':[], 'Temp_Arido':[], 'Temp_Cont':[], 'Rad_Ocean':[], 'Rad_Med_N':[], 'Rad_Med_S':[], 'Rad_Arido':[], 'Rad_Cont':[], 'Viento':[], 'Precio':[]}
#         base = {'Datetime':[], 'Demanda': [], 'Temp_Ocean':[], 'Temp_Resto':[],'Rad_Ocean':[], 'Rad_Resto':[], 'Viento':[], 'Precio':[]}
        base = {'Datetime':[], 'Demanda': [], 'Temp_Ocean_Cont':[], 'Temp_Med_Arido':[],'Rad_Ocean_Cont':[], 'Rad_Med_Arido':[], 'Viento':[], 'Precio':[]}
        for i in range (len(auxdemanda)):
            if i-n-2 <= 0:
                continue
            aux = {}
            aux['Datetime']=auxdemanda['Datetime'][i]
            aux['Demanda'] = np.mean(auxdemanda['Demanda'][i-(n+1):i-1])
#             if auxdemanda['Frío'][i] != 0:
#                 aux['Frío'] = auxdemanda['Frío'][i] - np.mean(auxdemanda['Frío'][i-(n-1):i])
#                 aux['Calor'] = 0
#             else:
#                 aux['Frío'] = 0
#                 aux['Calor'] = auxdemanda['Calor'][i] - np.mean(auxdemanda['Calor'][i-(n-1):i])
#             aux['Temp'] = auxdemanda['Temp'][i] - np.mean(auxdemanda['Temp'][i-(n+1):i])
#             aux['Radiacion'] =  auxdemanda['Radiacion'][i] - np.mean(auxdemanda['Radiacion'][i-(n+1):i])
            for zone in zonas_codigos.keys():
                aux['Temp_'+zone] = auxdemanda['Temp_'+zone][i] - np.mean(auxdemanda['Temp_'+zone][i-(n+1):i])
                aux['Rad_'+zone] =  auxdemanda['Rad_'+zone][i] - np.mean(auxdemanda['Rad_'+zone][i-(n+1):i])
            aux['Viento'] =  auxdemanda['Viento'][i] - np.mean(auxdemanda['Viento'][i-(n+1):i])
            aux['Precio'] = np.mean(auxdemanda['Precio'][i-(n+1):i-1])
            
            for name in base.keys():
                base[name].append(aux[name])
        
        base = pd.DataFrame(base)
        total[h] = base
#         print (total[h]['Datetime'][0])
    return total

# base_horaria = {}
# for h in range (24):
#     base_horaria[h] = df_horario[h][df_horario[h]['Datetime']>=threshold]
#     base_horaria[h].reset_index(drop=True,inplace=True)

base_horaria = build_base_horaria(df_horario)
base_horaria[0]

,Datetime,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-02-01,27167.976191,2.184176,0.482395,0.0,0.0,-0.003396,40.613571
1,2014-02-02,27252.523810,-4.617929,-1.638708,0.0,0.0,1.741033,40.819643
2,2014-02-03,27332.244048,-3.276210,-3.553070,0.0,0.0,-4.491425,40.580714
3,2014-02-04,27359.470238,-3.901654,-3.147135,0.0,0.0,2.405685,41.107143
4,2014-02-05,27397.976191,0.587969,1.637790,0.0,0.0,11.549490,41.065357
...,...,...,...,...,...,...,...,...
3616,2023-12-27,24663.095239,-4.170675,-3.667332,0.0,0.0,-7.032828,78.093929
3617,2023-12-28,24543.062500,-1.394251,-2.987254,0.0,0.0,-6.428653,78.728929
3618,2023-12-29,24497.086310,0.195078,-1.941345,0.0,0.0,-5.704132,77.881786
3619,2023-12-30,24479.803571,0.489342,-0.799822,0.0,0.0,-7.608623,78.584286


In [16]:
total_df = {}
cols = list (base_diaria.columns) + list (base_horaria[0].columns)[1:]
print (cols)
for h in range (24):
    auxbase = {x:[] for x in cols}
    for i in range (len(df_diario)):
        for name in cols[:len(df_diario.columns)]:
            auxbase[name].append(df_diario[name][i])
        for name in cols[len(df_diario.columns):]:
            auxbase[name].append(df_horario[h][name][i])
        i += 1
    total_df[h] = pd.DataFrame(auxbase)
        
total_df[0]

['Datetime', 'Day', 'Month', 'Week', 'F', 'L', 'M', 'X', 'J', 'V', 'S', 'IPC', 'Demanda', 'Temp_Ocean_Cont', 'Temp_Med_Arido', 'Rad_Ocean_Cont', 'Rad_Med_Arido', 'Viento', 'Precio']


,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-01-01,1,1,1,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,93373,24301.66667,6.236950,4.762810,0.0,0.0,18.565381,37.21
1,2014-01-02,2,1,1,0.019659,0.0,0.0,0.0,0.980341,0.0,0.0,93373,23484.00000,9.201634,8.185117,0.0,0.0,24.776614,22.13
2,2014-01-03,3,1,1,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,93373,26316.50000,11.000175,10.579728,0.0,0.0,23.791944,29.39
3,2014-01-04,4,1,1,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,93373,26518.50000,8.642548,9.609179,0.0,0.0,21.050676,36.54
4,2014-01-05,5,1,1,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,93373,26163.16667,3.840441,5.714173,0.0,0.0,28.805064,18.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2023-12-27,27,12,52,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,113308,23764.08333,2.214768,3.441935,0.0,0.0,10.034081,83.06
3648,2023-12-28,28,12,52,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,113308,24105.83333,4.735265,3.933591,0.0,0.0,10.283692,101.01
3649,2023-12-29,29,12,52,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,113308,24181.41667,6.130193,4.762047,0.0,0.0,10.691312,106.83
3650,2023-12-30,30,12,52,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,113308,24131.16667,6.216880,5.628986,0.0,0.0,8.431629,93.55


In [17]:
combined_df = pd.DataFrame()
for h in range (24):
    aux = total_df[h]
    aux['Datetime'] = pd.to_datetime(aux['Datetime'], format='%d-%m-%Y') + pd.DateOffset(hours=h)    
    aux['Datetime'] = pd.to_datetime(aux['Datetime'])
    combined_df = pd.concat((combined_df, aux))

combined_df = combined_df.sort_values(by='Datetime')

In [18]:
combined_df

,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-01-01 00:00:00,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,93373,24301.66667,6.236950,4.762810,0.0,0.0,18.565381,37.21
0,2014-01-01 01:00:00,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,93373,23365.33333,6.021042,4.755462,0.0,0.0,18.443398,30.57
0,2014-01-01 02:00:00,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,93373,21672.50000,5.949079,4.682017,0.0,0.0,18.401250,26.30
0,2014-01-01 03:00:00,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,93373,20192.00000,5.884401,4.685583,0.0,0.0,18.198554,26.18
0,2014-01-01 04:00:00,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,93373,19220.66667,6.094201,4.707938,0.0,0.0,18.100382,22.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,2023-12-31 19:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.0,113308,29135.66667,5.870583,8.220767,0.0,0.0,16.116332,109.07
3651,2023-12-31 20:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.0,113308,28703.50000,5.384606,8.026366,0.0,0.0,14.814726,101.69
3651,2023-12-31 21:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.0,113308,27423.16667,5.039849,7.261764,0.0,0.0,13.838921,92.69
3651,2023-12-31 22:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.0,113308,24613.91667,4.723137,7.499999,0.0,0.0,12.934626,86.82


In [19]:
base = {}
cols = list (base_diaria.columns) + list (base_horaria[0].columns)[1:]
print (cols)
for h in range (24):
    auxbase = {x:[] for x in cols}
    for i in range (len(base_diaria)):
        for name in cols[:len(base_diaria.columns)]:
            auxbase[name].append(base_diaria[name][i])
        for name in cols[len(base_diaria.columns):]:
            auxbase[name].append(base_horaria[h][name][i])
        i += 1
    base[h] = pd.DataFrame(auxbase)
        
base[0]

['Datetime', 'Day', 'Month', 'Week', 'F', 'L', 'M', 'X', 'J', 'V', 'S', 'IPC', 'Demanda', 'Temp_Ocean_Cont', 'Temp_Med_Arido', 'Rad_Ocean_Cont', 'Rad_Med_Arido', 'Viento', 'Precio']


,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-02-01,1,2,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.675067,0.999572,27167.976191,2.184176,0.482395,0.0,0.0,-0.003396,40.613571
1,2014-02-02,2,2,5,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.999586,27252.523810,-4.617929,-1.638708,0.0,0.0,1.741033,40.819643
2,2014-02-03,3,2,6,0.0,4.746262,0.000000,0.000000,0.000000,0.000000,0.000000,0.999601,27332.244048,-3.276210,-3.553070,0.0,0.0,-4.491425,40.580714
3,2014-02-04,4,2,6,0.0,0.000000,5.675067,0.000000,0.000000,0.000000,0.000000,0.999616,27359.470238,-3.901654,-3.147135,0.0,0.0,2.405685,41.107143
4,2014-02-05,5,2,6,0.0,0.000000,0.000000,7.055838,0.000000,0.000000,0.000000,0.999631,27397.976191,0.587969,1.637790,0.0,0.0,11.549490,41.065357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,2023-12-27,27,12,52,0.0,0.000000,0.000000,3.848015,0.000000,0.000000,0.000000,1.000026,24663.095239,-4.170675,-3.667332,0.0,0.0,-7.032828,78.093929
3617,2023-12-28,28,12,52,0.0,0.000000,0.000000,0.000000,3.848015,0.000000,0.000000,1.000017,24543.062500,-1.394251,-2.987254,0.0,0.0,-6.428653,78.728929
3618,2023-12-29,29,12,52,0.0,0.000000,0.000000,0.000000,0.000000,3.855166,0.000000,1.000009,24497.086310,0.195078,-1.941345,0.0,0.0,-5.704132,77.881786
3619,2023-12-30,30,12,52,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.855166,1.000000,24479.803571,0.489342,-0.799822,0.0,0.0,-7.608623,78.584286


In [20]:
combined_base = pd.DataFrame()
for h in range (24):
    aux = base[h]
    aux['Datetime'] = pd.to_datetime(aux['Datetime'], format='%d-%m-%Y') + pd.DateOffset(hours=h)    
    aux['Datetime'] = pd.to_datetime(aux['Datetime'])
    combined_base = pd.concat((combined_base, aux))

combined_base = combined_base.sort_values(by='Datetime')

In [21]:
combined_base

,Datetime,Day,Month,Week,F,L,M,X,J,V,S,IPC,Demanda,Temp_Ocean_Cont,Temp_Med_Arido,Rad_Ocean_Cont,Rad_Med_Arido,Viento,Precio
0,2014-02-01 00:00:00,1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,5.675067,0.999572,27167.976191,2.184176,0.482395,0.0,0.0,-0.003396,40.613571
0,2014-02-01 01:00:00,1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,5.675067,0.999572,24823.452381,2.178791,0.673189,0.0,0.0,1.138314,32.446071
0,2014-02-01 02:00:00,1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,5.675067,0.999572,23367.839286,2.027269,0.512497,0.0,0.0,1.520705,26.111429
0,2014-02-01 03:00:00,1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,5.675067,0.999572,22626.809524,1.773956,0.718430,0.0,0.0,1.418992,25.017857
0,2014-02-01 04:00:00,1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,5.675067,0.999572,22374.714285,1.491413,1.051654,0.0,0.0,1.395678,24.146786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,2023-12-31 19:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,31581.622024,-1.215103,-0.677267,0.0,0.0,0.903926,113.569643
3620,2023-12-31 20:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,32059.056548,-1.251963,0.014478,0.0,0.0,-0.213839,114.463929
3620,2023-12-31 21:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,31532.630952,-1.290650,-0.167071,0.0,0.0,-1.016181,107.638929
3620,2023-12-31 22:00:00,31,12,52,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,29049.639880,-1.418061,0.625601,0.0,0.0,-2.067612,97.948214


In [22]:
name_output = str(ndays_base)+'Days-3Temps-3Rads.csv'
combined_df.to_csv('./Base Inputs/No Variations/'+name_output, index=False)
combined_base.to_csv('./Base Inputs/Variations/'+name_output, index=False)